# Branch & Bound and Knapsack Lab

In [ ]:
!pip install pulp
!pip install gurobipy
!pip install gilp

In [ ]:
import pandas as pd
import gilp
from pulp import *
from google.colab import userdata

In [ ]:
!if [ ! -f "knapsack_data_1.csv" ]; then wget https://github.com/engri-1101/orie-3310/raw/refs/heads/main/labs/lab2/knapsack_data_1.csv; fi

Make sure that you have opened the Key menu, ticked Notebook access for each secret, and closed the key menu before running the next cell.

In [ ]:
options = {
"WLSACCESSID": userdata.get("WLSACCESSID"),
"WLSSECRET": userdata.get("WLSSECRET"),
"LICENSEID": int(userdata.get("LICENSEID")),
}

## Part 1: Branch and Bound Algorithm

Recall that the branch-and-bound algorithm (in addition to the simplex method) allows us to solve integer programs. Before applying the branch-and-bound algorithm to the knapsack problem, we will begin by reviewing some core ideas. Furthermore, we will identify a helpful property that will make branch-and-bound terminate quicker later in the lab!

### Question 1
What are the different ways a node can be fathomed during the branch and bound algorithm? Describe each.

#### Answer: (Hint: there are three possible ways.)

### Question 2
Suppose you have a maximization integer program and you solve its linear program relaxation. What does the LP-relaxation optimal value tell you about the IP optimal value? What if it is a minimization problem?



#### Answer (for maximization problems):


#### Answer (for minimization problems):

### Question 3
 Assume you have a maximization integer program with all integral coefficents in the objective function. Now, suppose you are running the branch and bound algorithm and come across a node with an optimal value to the current LP relaxation of 44.5. The current incumbent is 44. Can you fathom this node? Why or why not?



#### Answer:

### Question 4
(a) True or False: If the optimal value of the LP relaxation of the original program is integer, then you have found an optimal solution to your integer program. Explain why or why not this is true.


(b) True or False: If the optimal solution to the LP relaxation of the original program is integer, then you have found an optimal solution to your integer program. Explain why or why not this is true.


#### Answer to part (a):




#### Answer to part (b):

### Question 5
 True of False: If the LP is infeasible, then the IP is infeasible. Explain why or why not this is true.



#### Answer:

The next questions ask about the following branch-and-bound tree, in which, you may assume all of the variables are binary (i.e., restricted to either 0 or 1). For a given node of the tree, if the solution is not integral, the fractional $x_i$ that was used to branch is given. If the solution is integral, it is denoted *INT*. In the current iteration of branch-and-bound, you are looking at the node with the $z^*$.

<img src="https://github.com/engri-1101/orie-3310/blob/main/labs/lab2/part1_bnb_tree.png?raw=true" width="500"/>

### Question 6
 Can you determine if the integer program this branch-and-bound tree is for is a minimization or maximixation problem? If so, which is it, and how can you determine this?



#### Answer:.

*For **Q7-8**, you can assume integral coefficents in the objective function.*

### Question 7
 Is the current node (marked $z^*$) fathomed? Why or why not? If not, what additional constraints should be imposed for each of the next two nodes?

#### Answer:

### Question 8
 Consider the nodes under the current node (where $z = 16.3$). What do you know about the optimal value of these nodes? Why?


#### Answer:

## Part 2: The Knapsack Problem

In this lab, you will solve an integer program by branch-and-bound. The integer program to be solved will be a knapsack problem.

**Knapsack Problem:** We are given a collection of $n$ items, where each item $i = 1,\ldots,n$ has a weight $w_i$ and a value $v_i$. In addition, there is a given capacity $W$, and the aim is to select a maximum value subset of items that has a total weight at most $W$. Note that each item can be brought at most once.

$$\begin{align*}
\max \quad & \sum_{i=1}^n v_ix_i\\
\text{s.t.} \quad & \sum_{i=1}^n w_ix_i \leq W \\
& 0 \leq x_i \leq 1, \text{integer}, i = 1,\dots,n
\end{align*}$$

Consider the following data which we import from a CSV file:

In [ ]:
data = pd.read_csv('knapsack_data_1.csv', index_col=0)
data

and $W = 18$.

### Question 9
 Are there any items we can remove from our input to simplify this problem? Why? If so, replace `index` with the item number that can be removed in the code below. Hint: how many of each item could we possibly take?



#### Answer:

In [ ]:
# change the next line so index is equal to the number of the item that can be removed
index = -1
data = data.drop(index)
data

### Question 10
Does the IP optimal solution change if we remove item 7 from the knapsack input? Explain why or why not.


#### Answer:

### Question 11
 Does the LP relaxation's optimal value change if we remove item 7 from the knapsack input? Explain why or why not?


#### Answer:

In **Q10-11**, you should have found that removing these items removes feasible solutions from the linear program  but does not change the integer program. This is desirable as the gap between the optimal IP and LP values can become smaller. By adding this step, branch-and-bound may terminate sooner.

Recall that a branch and bound node can be fathomed if its bound is no better than the value of the best feasible integer solution found thus far. Hence, it helps to have a good feasible integer solution as quickly as possible (so that we stop needless work). To do this, we can first try to construct a good feasible integer solution by a reasonable heuristic algorithm before starting to run the branch and bound procedure.

In designing a heuristic for the knapsack problem, it is helpful to think about the value per unit weight for each item. We compute this value in the table below.

In [ ]:
data['value per unit weight'] = (data['value'] / data['weight']).round(2)
data

### Question 12
 Design a reasonable heuristic for the knapsack problem. Note a heuristic aims to find a decent solution to the problem (but is not necessarily optimal).



#### Answer:


### Question 13
 Run your heuristic on the data above to compute a good feasible integer solution. Your heuristic should generate a feasible solution with a value of 64 or better. If it does not, try a different heuristic (or talk to your TA!)



#### Answer:

We will now use the branch-and-bound algorithm to solve this knapsack problem! First, let us define a mathematical model for the linear relaxation of the knapsack problem.

### Question 14
Complete the model below.

In [ ]:
def Knapsack(table, capacity, integer = False):
    """Model for solving the Knapsack problem.

    Args:
        table (pd.DataFrame): A table indexd by items with a column for value and weight
        capcity (int): An integer-capacity for the knapsack
        integer (bool): True if the variables should be integer. False otherwise.
    """
    ITEMS = list(table.index)        # set of items
    v = table.to_dict()['value']     # value for each item
    w = table.to_dict()['weight']    # weight for each item
    W = capacity                     # capacity of the knapsack

    # define model
    m = LpProblem("knapsack", LpMaximize)

    # decision variables
    x = {}
    variable_category = "Integer" if integer else "Continuous"
    for i in ITEMS:
          x[i] = LpVariable('x_'+str(i), 0, 1, variable_category)

    # define objective function here
    m += sum(v[i]*x[i] for i in ITEMS)

    # Add a constraint that enforces that weight must not exceed capacity
    # recall that we add constraints to the model using +=
    raise ValueError('Replace this line with your code answer')
    return (m, x)  # return the model and the decision variables

In [ ]:
# You do not need to do anything with this cell but make sure you run it!
def solve(m):
    """Used to solve a model m."""
    solver = GUROBI(manageEnv=True, envOptions=options)
    m.solve(solver)
    solver.close()

    print('Objective =',value(m.objective))
    return [(var.name, value(var)) for var in m.variables()]

We can now create a linear relaxation of our knapsack problem. Now, `m` represents our model and `x` represents our decision variables.

In [ ]:
m, x = Knapsack(data, 18)

We can use the next line to solve the model and output the solution

In [ ]:
solve(m)

### Question 15
 How does this optimal value compare to the value you found using the heuristic integer solution?

#### Answer:

### Question 16
 Should this node be fathomed? If not, what variable should be branched on and what additional constraints should be imposed for each of the next two nodes?


#### Answer:

After constructing the linear relaxation model using `Knapsack(data, 18)` we can add additional constraints. For example, we can add the constraint $x_2 \leq 0$ and solve it as follows:

In [ ]:
m, x = Knapsack(data, 18)
m += (x[2] <= 0)
solve(m)

 **NOTE:** The line `m, x = Knapsack(data, 18)` resets the model `m` to the LP relaxation. All constraints from branching have to be added each time.

### Question 17
 Use the following cell to compute the optimal value for the other node you found in **Q16**.

In [ ]:
# Answer Q17
raise ValueError('Replace this line with your code answer')

### Question 18
What was the optimal value? Can this node be fathomed? Why? (Hint: In **Q13**, you found a feasible integer solution with value 64.)


#### Answer:

If we continue running the branch and bound algorithm, we will eventually reach the branch and bound tree below where the $z^*$ indictes the current node we are looking at.

<img src="https://github.com/engri-1101/orie-3310/blob/main/labs/lab2/part2_bnb_tree.png?raw=true" width="700"/>

### Question 19
 The node with $z = 64.857$ was fathomed. Why are we allowed to fathom this node? (Hint: think back to **Q3**)


#### Answer:

### Question 20
 Finish running branch and bound to find the optimal integer solution. Use a separate cell for each node you solve and indicate if the node was fathomed with a comment. (Hint: Don't forget to include the constraints further up in the branch and bound tree.)

In [ ]:
# copy paste this cell and add the extra constraints for each bnb node
m, x = Knapsack(data, 18)
# Add constraints here
######################
solve(m)

In [ ]:
raise ValueError('Replace this line with your code answer')

In [ ]:
raise ValueError('Replace this line with your code answer')

### Question 21
How many nodes in total did you have to explore while running the branch and bound algorithm? This should include the original LP relaxation.

#### Answer:

## Part 3: Geometry of Branch and Bound

We  will use the `gilp` package to viusualize branch and bound. We will give a quick overview of the tool. The function `bnb_visual` takes an `LP` and returns a visualization. It is assumed that every decision variable is constrained to be integer.  `bnb_visual` returns a series of figures for each node of the branch and bound tree. Let's look at a small 2D example:

$$\begin{align*}
\max \quad & 5x_1+ 8x_2\\
\text{s.t.} \quad & x_1 + x_2 \leq 6 \\
& 5x_1 + 9x_2 \leq 45 \\
& x_1, x_2 \geq 0, \quad \text{integral}
\end{align*}$$

In [ ]:
nodes = gilp.visualize.bnb_visual(gilp.examples.STANDARD_2D_IP)

In [ ]:
nodes[0].show()

Run the cells above to generate a figure for each node and look at the first node. At first, you will see the LP relaxation on the left and the root of the branch and bound tree on the right. The simplex path and isoprofit slider are also present.

### Question
Recall the root of a branch and bound tree is the unaltered LP relaxation. What is the optimal solution? (Hint: Use the objective slider and hover over extreme points).



#### Answer:

### Question
Assume that we always choose the variable with the minimum index to branch on if there are multiple options. Write down (in full) each of the LPs we get after branching off the root node.

#### Answer:

In [ ]:
nodes[1].show()

The outline of the original LP relaxation is still shown on the left. Now that we have eliminated some of the fractional feasible solutions, we now have 2 feasible regions to consider. The darker one is the feasible region associated with the current node which is also shaded darker in the branch and bound tree. The unexplored nodes in the branch and bound tree are not shaded in.

### Question
Which feasible solutions to the LP relaxation are removed by this branch?




#### Answer:

### Question
At the current (dark) node, what constraints will we add? How many feasible regions will the original LP relaxation be broken into?

#### Answer:

In [ ]:
nodes[2].show()

### Question
What is the optimal solution at the current (dark) node? Do we have to further explore this branch? Have we arrived at the optimal solution of the IP? Explain.


#### Answer:

### Question
Recall shaded nodes have been explored and the node shaded darker (and feasible region shaded darker) correspond to the current node and its feasible region. Nodes not shaded have not been explored. How many nodes have **NOT** yet been explored?


#### Answer:


### Question
How many nodes have a degree of one in the branch and bound tree? (That is, they are only connected to one edge). These nodes are called leaf nodes. What is the relationship between the leaf nodes and the remaining feasible region?


#### Answer:

In [ ]:
# Show the next two iterations of the branch and bound algorithm
nodes[3].show()
nodes[4].show()

### Question
At the current (dark) node, we added the constraint $x_1 \leq 1$. Why were the fractional solutions $1 < x_1 < 2$ not eliminated for $x_2 <= 3$?


#### Answer:

In [ ]:
# Show the next three iterations of the branch and bound algorithm
nodes[5].show()
nodes[6].show()
nodes[7].show()

### Question
 What constraints are enforced at the current (dark) node? Look at the third graph/tree. Why are there no feasible solutions at this node? (Hint: simplify the constraints that are enforced and think about feasibility)

#### Answer:

In [ ]:
nodes[8].show()

### Question
Are we done? If so, what nodes are fathomed and what is the optimal solution? Explain.


#### Answer:

Let's look at branch and bound visualization for an integer program with 3 decision variables!

In [ ]:
nodes = gilp.bnb_visual(gilp.examples.VARIED_BRANCHING_3D_IP)

In [ ]:
# Look at the first 3 iterations
nodes[0].show()
nodes[1].show()
nodes[2].show()

Let's fast-forward to the final iteration of the branch and bound algorithm.

In [ ]:
nodes[-1].show()

### Question
Consider the feasible region that looks like a rectangular box with one corner point at the origin. What node does it correspond to in the tree? What is the optimal solution at that node?


#### Answer:

### Question
How many branch and bound nodes did we explore? What was the optimal solution? How many branch and bound nodes would we have explored if we knew the value of the optimal solution before starting branch and bound?


#### Answer: